# Стартапы

## 1. Загрузка и подготовка данных

### 1.1. Чтение данных

In [2]:
# Импортируем необходимые библиотеки
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
import seaborn as sns

from scipy import stats as st

from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import train_test_split

from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

import phik
import os

pd.set_option('display.float_format', '{:.2f}'.format)

In [3]:
sample_submit = pd.read_csv('kaggle_startups_sample_submit_28062024.csv')
test = pd.read_csv('kaggle_startups_test_28062024.csv')
train = pd.read_csv('kaggle_startups_train_28062024.csv')

In [4]:
# Проверим размеры получившихся датасетов
print(sample_submit.shape)
print(test.shape)
print(train.shape)

(13125, 2)
(13125, 11)
(52516, 13)


### 1.2. Изучение общей информации о датасетах

In [5]:
# Создадим функцию для просмотра общей информации о датасетах
def overview_data(data):
    dash = '-' * 50

    print(dash)
    print('Основная информация о данных')
    print(dash)
    print(data.info())
    print('\n')

    print(dash)
    print('Первые строки данных')
    print(dash)
    print(data.head())
    print('\n')

    print(dash)
    print('Описательная статистика')
    print(dash)
    print(data.describe())
    print('\n')

    print(dash)
    print('Пропущенные значения')
    print(dash)
    print(data.isna().sum())
    print('\n')

    print(dash)
    print('Дубликаты')
    print(dash)
    print(data.duplicated().sum())
    print('\n')

In [7]:
# Выведем общую информацию о датасете train
overview_data(train)

--------------------------------------------------
Основная информация о данных
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52516 entries, 0 to 52515
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   name               52515 non-null  object 
 1   category_list      50051 non-null  object 
 2   funding_total_usd  42447 non-null  float64
 3   status             52516 non-null  object 
 4   country_code       47014 non-null  object 
 5   state_code         45753 non-null  object 
 6   region             46157 non-null  object 
 7   city               46157 non-null  object 
 8   funding_rounds     52516 non-null  int64  
 9   founded_at         52516 non-null  object 
 10  first_funding_at   52516 non-null  object 
 11  last_funding_at    52516 non-null  object 
 12  closed_at          4917 non-null   object 
dtypes: float64(1), int64(1), object(11)

In [8]:
# Выведем общую информацию о датасете train
overview_data(test)

--------------------------------------------------
Основная информация о данных
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13125 entries, 0 to 13124
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   name               13125 non-null  object 
 1   category_list      12534 non-null  object 
 2   funding_total_usd  10547 non-null  float64
 3   country_code       11743 non-null  object 
 4   state_code         11430 non-null  object 
 5   region             11536 non-null  object 
 6   city               11538 non-null  object 
 7   funding_rounds     13125 non-null  int64  
 8   first_funding_at   13125 non-null  object 
 9   last_funding_at    13125 non-null  object 
 10  lifetime           13125 non-null  int64  
dtypes: float64(1), int64(2), object(8)
memory usage: 1.1+ MB
None


--------------------------------------------------
Первые строки да

### 1.3. Предобработка данных

In [9]:
# Посмотрим количество уникальных значений в датасете train
train.nunique()

name                 52515
category_list        22105
funding_total_usd    15551
status                   2
country_code           134
state_code             300
region                1036
city                  4477
funding_rounds          19
founded_at            5402
first_funding_at      4603
last_funding_at       4305
closed_at             3008
dtype: int64

In [10]:
# Рассмотрим уникальные значения в столбце name
print(list(train['name'].unique()))

['Lunchgate', 'EarLens', 'Reviva Pharmaceuticals', 'Sancilio and Company', 'WireTough Cylinders', 'Connected Sports Ventures', 'Attensity', 'Mesh Networks', 'AngioScore', 'Vidatronic', 'Indie Vinos', 'Seven Seas Water', 'Iberic Premium', 'BuildersCloud', 'Purewine', 'paack', 'Salesjob', 'Avi Networks Inc', 'The Glassbox', 'Cipher Surgical', 'Fisoc', 'NodePing', 'Human Performance Integrated Systems', 'Vputi', 'Savasti', 'Brightree', 'The Center from Health Promotion', 'Blurr', 'SmashFly', 'NewWorld Communications', 'MedServe', 'Perpay Inc', 'Plex Systems', 'Back to the Roots', 'Evargrah Entertainment Group', 'Bibulu', 'Crushpath', 'WP Rocket Holdings', 'Yarwoods Martial Arts', 'Algama', 'Evozym Biologics', 'Experfy', 'Safe Cash', 'Nyheter24-Gruppen', 'Pro-Cure Therapeutics', 'iOrder Fresh', 'Tivorsan Pharmaceuticals', 'Infiniu', 'Diagnostic Healthcare', 'Unreasonable Adventures', 'Tripalocal', 'Ferric Semiconductor', 'TaiMed Biologics', 'Domosite', 'Lucky Pai', 'Marco Polo Project', 'L

In [11]:
# Рассмотрим уникальные значения в столбце category_list
print(list(train['category_list'].unique()))

['Online Reservations|Restaurants', 'Manufacturing|Medical|Medical Devices', 'Biotechnology', 'Health Care', 'Manufacturing', 'Mobile', 'Analytics|Business Analytics|Social CRM|Social Media Monitoring|Software', 'Software', 'Semiconductors', 'Clean Technology', nan, 'Nanotechnology|Technology|Wine And Spirits', 'Customer Service|Delivery|Retail', 'Analytics|Cloud Data Services|Computers|Networking|Software', 'Banking|Finance|Loyalty Programs', 'Messaging', 'Consumers', 'Generation Y-Z|Photo Sharing|Social Media', 'College Recruiting|Enterprise Software|Recruiting|SaaS|Social Recruiting|Software', 'Technology', 'Finance|Financial Services|Payments', 'Cloud Computing|Enterprise Resource Planning|Manufacturing|Software', 'DIY|Environmental Innovation', 'Entertainment|Games', 'Internet|Pets|Services', 'Advertising|Sales and Marketing', 'Fitness|Sports', 'Consumer Goods|Food Processing|Organic Food', 'Analytics|Big Data|Business Intelligence|Marketplaces', 'Moneymaking', 'Media', 'Apps', 'E

In [20]:
# Рассмотрим уникальные значения в столбце funding_total_usd
print(list(train['funding_total_usd'].unique()))

[np.float64(828626.0), np.float64(42935019.0), np.float64(35456381.0), np.float64(22250000.0), np.float64(nan), np.float64(4300000.0), np.float64(90000000.0), np.float64(42000000.0), np.float64(1250500.0), np.float64(20000000.0), np.float64(107239.0), np.float64(1100000.0), np.float64(335000.0), np.float64(30000.0), np.float64(33000000.0), np.float64(425000.0), np.float64(4475451.26288144), np.float64(50990429.0), np.float64(350000.0), np.float64(50000.0), np.float64(10000000.0), np.float64(2370000.0), np.float64(9000000.0), np.float64(150000000.0), np.float64(8000000.0), np.float64(2500000.0), np.float64(86500000.0), np.float64(2000000.0), np.float64(12032712.0), np.float64(25480000.0), np.float64(1200.0), np.float64(600000.0), np.float64(1500000.0), np.float64(1200000.0), np.float64(2238535.0), np.float64(1000000.0), np.float64(96325.0), np.float64(1503411.0), np.float64(850000.0), np.float64(22000000.0), np.float64(700000.0), np.float64(40000000.0), np.float64(3000.0), np.float64(19

In [21]:
# Рассмотрим уникальные значения в столбце status
print(list(train['status'].unique()))

['operating', 'closed']


In [23]:
# Рассмотрим уникальные значения в столбце country_code
print(list(train['country_code'].unique()))

['CHE', 'USA', nan, 'GBR', 'RUS', 'ESP', 'FRA', 'SWE', 'IND', 'KOR', 'AUS', 'TWN', 'CHN', 'HUN', 'CAN', 'FIN', 'BRA', 'EGY', 'PRT', 'ISR', 'CHL', 'SVK', 'NLD', 'ARG', 'ARE', 'JPN', 'IRL', 'DEU', 'LBN', 'EST', 'IDN', 'BEL', 'MEX', 'DNK', 'ITA', 'GRC', 'ROM', 'TUR', 'SGP', 'SAU', 'MYS', 'NGA', 'QAT', 'VNM', 'BGR', 'GHA', 'UGA', 'JOR', 'AUT', 'PAK', 'NZL', 'HKG', 'ZAF', 'LVA', 'NOR', 'HRV', 'VEN', 'PER', 'LUX', 'SVN', 'POL', 'PHL', 'NIC', 'CZE', 'ISL', 'PAN', 'URY', 'BGD', 'MCO', 'GTM', 'THA', 'TTO', 'UKR', 'KEN', 'CRI', 'DZA', 'LTU', 'CYM', 'MUS', 'BLR', 'SLV', 'COL', 'MLT', 'PSE', 'BLZ', 'CYP', 'MMR', 'MKD', 'KHM', 'GIB', 'SRB', 'DOM', 'BWA', 'BLM', 'PRY', 'BMU', 'TAN', 'LIE', 'GRD', 'UZB', 'GEO', 'SEN', 'ALB', 'ECU', 'MOZ', 'LKA', 'LAO', 'TUN', 'HND', 'ZWE', 'BAH', 'MDA', 'MAF', 'BHR', 'AZE', 'MAR', 'KWT', 'IRN', 'GGY', 'KNA', 'PRI', 'OMN', 'KAZ', 'JAM', 'NPL', 'ARM', 'TGO', 'RWA', 'BRN', 'JEY', 'SOM', 'CMR', 'MNE', 'SYC', 'ZMB']


In [24]:
# Рассмотрим уникальные значения в столбце state_code
print(list(train['state_code'].unique()))

['25', 'CA', nan, 'VA', 'NJ', 'TX', 'OR', 'FL', 'WA', 'NY', 'C7', 'AZ', '91', 'GA', 'MA', 'PA', 'MI', '56', 'MN', 'A8', 'DE', '26', 'Q5', '7', 'RI', '11', 'C5', 'CO', '3', '23', '1', 'ON', 'IL', 'OH', 'K2', '2', '14', '30', 'H9', 'AL', '29', '6', 'NC', '5', '12', '22', 'MD', '48', 'ND', '40', 'K4', 'VT', '16', '4', 'UT', 'IN', '8', '9', '18', 'CT', '13', 'NV', '10', 'B8', '27', '17', 'IA', 'WI', 'L9', 'NM', 'B7', 'MO', 'BC', '21', 'F7', 'ME', 'NH', '35', 'NS', 'C1', 'C3', 'SC', '20', 'TN', 'A6', '51', 'I7', 'LA', 'KS', '42', 'N7', 'U8', 'I2', 'AR', '19', 'NE', '41', 'QC', 'AB', '37', '15', '97', 'V2', 'D2', 'DC', 'A7', 'C9', 'F8', 'Y2', 'B9', 'J7', 'M9', 'G2', 'H3', 'ID', 'NB', '28', 'V8', '34', 'B5', 'F2', 'A9', 'Z7', '53', 'H8', 'MT', 'MB', 'KY', '61', '60', 'B4', 'H7', '82', 'A2', 'C8', 'L3', 'B3', 'D9', 'M3', 'MS', '31', '33', '52', 'HI', 'AK', '78', '72', 'Q4', 'Y4', 'WY', '32', 'E6', 'Q3', 'F9', '24', 'OK', 'X4', '81', 'A1', '86', 'P4', 'E7', 'P9', '73', 'W5', 'D4', 'E9', 'A4', '

In [25]:
# Рассмотрим уникальные значения в столбце region
print(list(train['region'].unique()))

['Zurich', 'SF Bay Area', nan, 'VA - Other', 'Newark', 'Houston', 'Austin', 'Portland, Oregon', 'Tampa', 'Seattle', 'New York City', 'Coventry', 'Phoenix', 'RUS - Other', 'Atlanta', 'Los Angeles', 'Boston', 'Philadelphia', 'Detroit', 'Barcelona', 'Minneapolis', 'Paris', 'Dover', 'Stockholm', 'York', 'Delhi', 'Providence', 'Seoul', 'Altrincham', 'Denver', 'Taiwan', 'Shanghai', 'Melbourne', 'HUN - Other', 'Ft. Lauderdale', 'Ottawa', 'Chicago', 'Akron - Canton', 'London', 'Dallas', 'Washington, D.C.', 'Ontario - Inland Empire', 'AUS - Other', 'FIN - Other', 'Porto Alegre', 'Shenzhen', 'Huntsville', 'Leganes', 'Alexandria', 'Charlotte', 'Porto', 'Tel Aviv', 'Santiago', 'Beijing', 'Neuilly-sur-seine', 'Baltimore', 'Miami', 'Moscow', 'San Diego', 'San Antonio', 'Bratislava', 'ItÃ¡', 'Fargo', 'Amsterdam', 'Buenos Aires', 'Dubai', 'Madrid', 'Tokyo', 'Plymouth', 'Palm Beaches', 'VT - Other', 'Dublin', 'IL - Other', 'Berlin', 'Beirut', 'Salt Lake City', 'Tallinn', 'Jakarta', 'South Bend', 'Antwe

In [26]:
# Рассмотрим уникальные значения в столбце city
print(list(train['city'].unique()))

['Zürich', 'Redwood City', 'San Jose', nan, 'Bristol', 'Princeton', 'Houston', 'Fremont', 'College Station', 'Portland', 'Tampa', 'Bellevue', 'Sunnyvale', 'New York', 'Coventry', 'Austin', 'Phoenix', 'Krasnoyarsk', 'Lawrenceville', 'Los Angeles', 'Concord', 'Bellaire', 'Philadelphia', 'Troy', 'Oakland', 'Barcelona', 'San Francisco', 'Maple Grove', 'Paris', 'Lewes', 'Boston', 'Stockholm', 'York', 'Delhi', 'Providence', 'Seoul', 'Altrincham', 'Boulder', 'Taipei', 'Shanghai', 'Melbourne', 'Kecskemét', 'Pembroke Pines', 'Ottawa', 'Itasca', 'Tallmadge', 'Oxford', 'Sachse', 'Kirkland', 'Reston', 'Loma Linda', 'Botany', 'Joensuu', 'Porto Alegre', 'Somerville', 'Shenzhen', 'London', 'Palo Alto', 'Huntsville', 'Leganés', 'Rancho Cucamonga', 'Alexandria', 'Charlotte', 'Porto', 'Ramat Gan', 'Santiago', 'Beijing', 'Sausalito', 'Evanston', 'Chicago', 'Mountain View', 'Neuilly-sur-seine', 'Columbia', 'South San Francisco', 'Tel Aviv', 'Ann Arbor', 'Miami', 'Moscow', 'San Diego', 'Cypress', 'San Anto

In [27]:
# Рассмотрим уникальные значения в столбце funding_rounds
print(list(train['funding_rounds'].unique()))

[np.int64(2), np.int64(4), np.int64(3), np.int64(1), np.int64(5), np.int64(7), np.int64(9), np.int64(6), np.int64(8), np.int64(10), np.int64(11), np.int64(12), np.int64(15), np.int64(13), np.int64(16), np.int64(17), np.int64(14), np.int64(18), np.int64(19)]


In [28]:
# Рассмотрим уникальные значения в столбце founded_at
print(list(train['founded_at'].unique()))

['2009-10-17', '2005-01-01', '2006-01-01', '2004-01-01', '2010-05-12', '2011-04-16', '2000-01-01', '2003-01-01', '2010-01-01', '2009-01-01', '1997-01-01', '2013-07-26', '2012-01-01', '2014-01-01', '2013-04-06', '2013-01-01', '2009-05-29', '2011-05-17', '2010-03-05', '2012-04-26', '2015-01-01', '2007-03-11', '2002-04-21', '2007-01-01', '1999-05-04', '2007-01-15', '2014-02-01', '1995-01-01', '2009-03-01', '2013-08-13', '2013-02-01', '2011-04-01', '2011-01-01', '2013-09-28', '2013-11-27', '2007-04-11', '2006-04-26', '2011-03-27', '2012-11-01', '2011-07-12', '2005-10-09', '2011-05-01', '2014-05-01', '2001-01-01', '2011-03-22', '2008-10-21', '2008-01-01', '2009-02-22', '2013-11-24', '2011-02-22', '2009-03-11', '2007-08-01', '2007-11-01', '2007-06-09', '2013-05-13', '2014-04-01', '1973-01-01', '2010-02-12', '1996-01-01', '2011-06-14', '2013-01-23', '2010-09-14', '2010-07-01', '1999-01-01', '2011-10-26', '2010-04-19', '2014-09-01', '2011-01-03', '2005-11-01', '2006-08-24', '1992-12-23', '2013